In [2]:
##########################################################################################

!pip install PyCryptoDomex elgamal

# You only need to run this line once to install the library and then comment it out

##########################################################################################

from multiprocessing import Pipe, Process
from elgamal.elgamal import Elgamal, PublicKey, CipherText
from random import randint, getrandbits

# lengh of public key (bytes)
key_length = 32 
# length of item (bits)
item_bit_length = 5
# length of mask (lambda)
mask_bit_length = 6
# number of items
num_item = 8

##########################################################################################

# Alice's process
def Alice(end_a): 
  
    # set held by Alice
    item_set = [28, 15, 7, 3, 23, 20, 31, 19]

    # intersection of Bob's and Alice's sets
    intersection = set()

    # iterate over each item of Alice's
    for item in item_set:
        
        # indicator if x equals some item in Bob's set
        b = False

        # iterate over each item of Bob's
        for i in range(num_item):
            
            # check whether item is in Bob's set using private equality test
            for j in range(item_bit_length):
            
                # TODO: get the j-th bit of item
                bit = item >> (item_bit_length-1-j) & 1

                # TODO: generate a pair of public, secret keys of Elgamal
                pk, sk = Elgamal.newkeys(key_length)

                # TODO: generate a plausible random public key
                random_y = randint(1, pk.p-1)

                # TODO: construct the message of format
                # (i, j, random_y, pk.y, pk.p, pk.g) if bit is 1 or (i, j, pk.y, random_y, pk.p, pk.g) if bit is 0
                if bit:
                    msg = ",".join([str(t) for t in[i, j, random_y, pk.y, pk.p, pk.g]])
                    
                else:
                    msg = ",".join([str(t) for t in[i, j, pk.y, random_y, pk.p, pk.g]])

                # send message to Bob
                end_a.send(msg)

                # receive message from Bob
                masks = end_a.recv().split(',')

                # TODO: construct the ciphertext corresponding to bit
                cipher = CipherText(int(masks[bit*2]), int(masks[bit*2+1]))

                # TODO: decrypt the ciphertext to get the message
                mask = int(Elgamal.decrypt(cipher, sk))

                # TODO: update Alice's overall mask by XOR with mask               
                if j == 0:
                    alice_mask = mask
                else:
                    alice_mask ^=  mask

                if j == item_bit_length-1:
                    bob_mask = int(end_a.recv())
                    # TODO: check if Alice's and Bob's overall masks agree if it is the last bit
                    b = alice_mask == bob_mask

            if b:
                intersection.add(item)
                break

    # indicate that the protocol is over 
    end_a.send('done!')
    print('Alice and Bob\'s intersection is:', intersection)


##########################################################################################


# Bob's process
def Bob(end_b):

    # set of items held by Bob
    item_set = [14, 23, 2, 24, 7, 5, 17, 20]
    
    while True:
          
          # receive Alice's request
          msg = end_b.recv()

          # receive the signal that the protocol is over
          if msg == 'done!':
              break
          
          # retrieve fields from the request
          fields = msg.split(',')
          i, j, y0, y1, p, g = int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3]), int(fields[4]), int(fields[5])
            
          # identify the item Alice is checking against 
          item = item_set[i]
            
          # reset bob's mask if Alice is checking a new item
          if j == 0:
              bob_mask = 0
            
          # identify the bit Alice is checking against
          bit = item >> (item_bit_length-1-j) & 1
            
          # generate two masks for this bit
          masks = [getrandbits(mask_bit_length), getrandbits(mask_bit_length)]
          
          # update Bob's overall mask by XOR 
          bob_mask ^= masks[bit]
            
          # encryption for the first mask
          pk = PublicKey(p, g, y0)
          mask0 = Elgamal.encrypt(bytes(str(masks[0]), 'utf-8'), pk)
    
          # encryption for the second mask 
          pk = PublicKey(p, g, y1)
          mask1 = Elgamal.encrypt(bytes(str(masks[1]), 'utf-8'), pk)
       
          # send response back to Alice
          msg = ','.join([str(t) for t in [mask0.a, mask0.b, mask1.a, mask1.b]])
          end_b.send(msg)
            
          # send Bob's overall mask if it is the last bit      
          if j == item_bit_length - 1:
              end_b.send(str(bob_mask))

##########################################################################################
if __name__ == "__main__":

    end_a, end_b = Pipe()
    
    # start Alice's process
    alice_p = Process(target=Alice, args=(end_a,))
    alice_p.start()

    # start Bob's process
    Bob(end_b)

    # wait for Alice's process to end
    alice_p.join()

Alice and Bob's intersection is: {23, 20, 7}
